In [1]:
import glob 
import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import cv2
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
import csv 

In [2]:
def plotly_visualization(x, y, title=''):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=y, mode='lines', name='ECG'))
    fig.add_trace(go.Scatter(x=x, y=y[x], mode='markers', marker=dict(symbol='x', size=8, color='red'), name='R-peaks'))

    fig_width = 800 
    fig_height = 250  
    fig.update_layout(
        title=title,
        yaxis= dict(autorange='reversed'),
        width=fig_width,
        height=fig_height
    )

    fig.show()

In [3]:
def img_visualization(image, title='', color=''):
    if color=='gray':
        fig = px.imshow(image, color_continuous_scale='gray')
    else:
        fig = px.imshow(image)
    fig.update_layout(
        title=title,
        width=800,
        height=200,
        coloraxis_showscale=False
    )
    fig.show()

#### Checking for the color space (if YCbCr then color mapping is required)

In [4]:
# def find_color_space(dcm):
#     # Find through meta data
#     transfer_syntax_uid = dcm.file_meta.TransferSyntaxUID
#     if transfer_syntax_uid == ('1.2.840.10008.1.2.4.50' or '1.2.840.10008.1.2.4.51' or '1.2.840.10008.1.2.4.57' or '1.2.840.10008.1.2.4.70'):
#         # print('YCbCr')
#         return 'YCbCr'
#     else:
#         # print('RGB')
#         return 'RGB'

##### Detecting R-peaks and returning their spatial information 

In [5]:
def detect_peaks(y_coordinates):
    # Detect R-peaks

    peaks = list()
    # last_non_none = next((item for item in reversed(y_coordinates) if item is not None), None)
    y_coordinates_non_none = y_coordinates[y_coordinates != None] 
    # as its a negative number I want to keep the actual low so the negative threshold will be high
    threshold_value =  -int((y_coordinates_non_none.max() + y_coordinates_non_none.min())/2)
    # assigning a high value that doesn't affect peaks as find_peaks() input shouldn't have none vals
    y_coordinates[y_coordinates == None] = 1000

    #indicating fare x distance between each R peak based on the range of x
    xrange_ECG = np.where(y_coordinates!=1000)[0]
    x_distance = int((xrange_ECG[-1] - xrange_ECG[0])/4)
    
    x_peaks, _ = find_peaks(-y_coordinates, distance=x_distance)
    # threshold to filter out peaks below a certain amplitude
    x_peaks = [x for x in x_peaks if -y_coordinates[x] > threshold_value]

    y_coordinates[y_coordinates == 1000] = None

    # plotly_visualization(x_peaks, y_coordinates, "ECG with Detected R-peaks")

    for x_peak in x_peaks:
        peaks.append((x_peak, y_coordinates[x_peak]))

    return peaks

##### Converting to 1D vector

In [6]:
def vector1D(image2D):
 
    # For each column in the image, find the y-coordinate of the curve
    y_coordinates = list()
    for x in range(image2D.shape[1]):
        column = image2D[:, x]
        indices = np.where(column > 0)
        if indices[0].size > 0:
            y_coordinate = int(np.min(indices))  # Max of the non-background pixels positions in the column
            y_coordinates.append(y_coordinate)
        else:
            y_coordinates.append(None)

    y_coordinates = np.array(y_coordinates)

    return y_coordinates


In [7]:
def green_line_extractor(image):
   
   # if color_map == 'YCbCr':
   image = cv2.cvtColor(image, cv2.COLOR_YCrCb2RGB)

    # converting to hsv
   hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

   # Define the range for the green color
   lower_green = np.array([40, 40, 40])
   upper_green = np.array([80, 255, 255])

   # Create a mask for the green color
   mask = cv2.inRange(hsv_image, lower_green, upper_green)

   # Set non-green pixels to 0 and green pixels to a specific value
   # filtered_image = cv2.bitwise_and(image, image, mask=mask)

   return mask


In [8]:
def extraction(image):

    mask = green_line_extractor(image)
    if(mask.sum()==0):
        return []
    # img_visualization(mask, "Exteracted ECG line from image", 'gray')
    # cv2.imwrite(f'{name}.jpg', filtered_image)
    vector = vector1D(mask)
    coordinates = detect_peaks(vector)

    return coordinates 


##### Extracting ECG for each cardiac slice

In [9]:
def cycle_indentifier(image, coordinates):
    
    indices = list()
    next_temporal = 0

    for num, coordinate in enumerate(coordinates):
        x_axis = coordinate[0]
        expected_h = coordinate[1]

        heights = list()
        for x in range (next_temporal, image.shape[0]):
            test = image[x, :, :, :]
            signal = green_line_extractor(test)
            for i in range(signal.shape[0]):
                if (signal[i,x_axis]) > 0:
                    heights.append(i)

            if (heights):
                if (expected_h == int(np.array(heights).min())) or (expected_h in heights):
                    print(f'{num+1} cycle is completed at time index {x}')
                    next_temporal = x
                    indices.append(x)
                    break

    return indices
                

#### Percentize Label

In [10]:
# def label(indicies, images):
#     r2r_distance = min(indicies[:-1] - indicies[1:])
    
#     for image in images:


In [11]:
images = list()
updated_paths = list()
dict = {}
# temporal_indecies = list()
paths = glob.glob(os.path.join('d:/University/1-Calgary/experiments/1 - reading images/images/', '*' ))
dcms = [pydicom.dcmread(path) for path in paths]
updated_paths= [path.split('\\')[-1] for path in paths]

with open('firstI.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'indicies'])

    for i,dcm in enumerate(dcms):
        img = dcm.pixel_array
        if len(img.shape)<4:
            # updated_paths.remove(updated_paths[i])
            continue
        offset = int(8/9*img.shape[1])
        # images.append(img[:,offset:,:,:])
        image = img[:,offset:,:,:]
        
        coordinates  = extraction(image[-1,:,:,:].squeeze())
        print(coordinates)

        if len(coordinates) == 0:
            continue

        s = cycle_indentifier(image, coordinates)
        # temporal_indecies.append(s)

        # dict[f'{updated_paths[i]}'] = s
        writer.writerow([f'{updated_paths[i]}', s])     
                                     
        print('\n')


[(57, 6), (251, 24), (448, 33)]
1 cycle is completed at time index 1
2 cycle is completed at time index 40
3 cycle is completed at time index 79


[(62, 26), (264, 26), (473, 25)]
1 cycle is completed at time index 1
2 cycle is completed at time index 24
3 cycle is completed at time index 47


[(65, 24), (565, 7)]
1 cycle is completed at time index 2
2 cycle is completed at time index 57


[(58, 23), (257, 26), (461, 28)]
1 cycle is completed at time index 1
2 cycle is completed at time index 38
3 cycle is completed at time index 75


[(68, 27), (267, 26), (459, 27), (652, 33)]
1 cycle is completed at time index 2
2 cycle is completed at time index 23
3 cycle is completed at time index 43
4 cycle is completed at time index 63


[(58, 21), (234, 28), (410, 29)]
1 cycle is completed at time index 1
2 cycle is completed at time index 19
3 cycle is completed at time index 38


[(74, 30), (187, 27), (466, 6)]
1 cycle is completed at time index 2
2 cycle is completed at time index 14
3 cycle

In [12]:
# dict = {}
# temporal_indecies = list()
# for i, image in enumerate(images):
#     # color_map = find_color_space(dcm)

#     print(i)
#     # img_visualization(image[-1,:,:,:])
#     # pass the 3D image
#     coordinates  = extraction(image[-1,:,:,:].squeeze())
#     print(coordinates)

#     if len(coordinates) == 0:
#         continue
#     s = cycle_indentifier(image, coordinates)
#     temporal_indecies.append(s)

#     dict[f'{updated_paths[i]}'] = s

#     print('\n')
 

In [13]:
# import csv 

# # dict = {}
# # for temporal_index, name in zip(temporal_indecies, updated_paths):
# #     dict[f'{name}'] = temporal_index

# with open('first.csv', 'w') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['name', 'indicies'])
#     for k,v in dict.items():
#         writer.writerow([k, v])
        


In [14]:
# s = pydicom.dcmread('d:/University/1-Calgary/experiments/1 - reading images/ii/7VIQ9Y0R').pixel_array
# s.shape